In [1]:
from dotenv import load_dotenv

load_dotenv()

False

In [6]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [20]:
from langchain.prompts import PromptTemplate

template = """
    Context: {context}

    Question: {question}
 """

prompt = PromptTemplate.from_template(template)

In [21]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/Users/karen/Desktop/MachineLearning/RAG_example/skills-based-cv.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='ASHLEY GILL\n3 Lappage Court Telephone: 01882 652349  \nTyler Green, Bucks. Mobile: 07717 121824 \nHP8 4JD Email: ashleygill2023@gotmail.com\nPersonal Details\nSummary\n• Business studies with Spanish undergraduate.\n• Ability to speak French and Spanish.\n• Extensive business experience including an internship with Top Choice Holidays.\nEducation And Qualifications\n2008 – present Buckinghamshire Edge University \nBA International Business Studies with Spanish (expected 2:1)\n• Study semester at The University of Valloid (Spain).\n• Six-month work placement in Madrid.\n• Relevant modules included: Business Planning; Sales Promotion and\nMarketing; and Business Operations Management.\n2000 – 2007 Freebridge School\nA-Levels: Business Studies (B), French (C)\n8 GCSEs including Maths, English, Spanish and French\nWork History\n2008 – 2011 Buckinghamshire Edge University - Librarian/tour guide\n• General administrative and customer service roles.\n2011 (Feb–Aug) Au

In [22]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

retriever = vectorstore.as_retriever()

In [23]:
questions = [
    "What are his research interests?",
    "Does he have teaching experience?",
    "What are his hobbies?"
]

for question in questions:
    retrieval_query = f"Query the vector store for: {question}"
    
    retrieval_response = retriever.invoke(retrieval_query)
    
    formatted_query_for_model = prompt.format(context=retrieval_response, question=question)

    final_model_response =  model.invoke(formatted_query_for_model)

    parsed_response = parser.parse(final_model_response)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

Question: What are his research interests?
Answer: The provided CV does not mention Ashley Gill's research interests. The CV focuses on his education, work experience, skills, and achievements, but it does not provide information about his research interests or any academic projects he may have worked on during his studies.

Question: Does he have teaching experience?
Answer: According to the provided context, Ashley Gill has voluntary teaching experience in Mexico and Spain. This is mentioned in the "Voluntary Experience" section of his CV:

```
2007/2011 Teaching English in Mexico/Spain
```

Question: What are his hobbies?
Answer: According to the document, Ashley Gill's interests include:

* Active member of University Business Club
* Winner of the 'Bucks Best Business Pitch' award in 2010 Enterprise week, judged by Michael Eavis.
* Teaching English in Mexico/Spain (voluntary experience)

There is no specific mention of his hobbies.

